# Welcome to FOSSIL!
                                                                                                                   
                                                                                                                   
## Overview
This tool uses neural networks alongside the powerful CEGIS architecture to automatically synthesise sound Lyapunov functions of Barrier Certificates for any N-dimensional system. 

To use this tool please change the python code in the corresponding cells to input your required dynamical system and domain, then use the correspoding cell to synthise a [Lyapunov function](#lyap-synth) of [Barrier function](#barrier-synth) respectively.

If you would like access to more settings, such as , please use the [advanced-playground](FOSSIL-advanced-playground.ipynb). To see benchmarks and further examples refer to [benchmarks](FOSSIL-benchmarks.ipynb).

In [1]:
# % Imports
import sys

sys.path.append('..')
from experiments.playground_utils import *
#sys.path.append(".")

### Synthesising Lyapunov Functions

First, we demonstrate synthesising Lyapunov functions. Consider the following dynamical system:

$$ \begin{cases} \dot{x} = -x + xy \\ \dot{y} = - y \end{cases}$$


This system can be instantied as follows, with $x_0 = x $ and $x_1 = y$:


In [14]:

N_Dimensions = 2

x, y = initialise_states(N_Dimensions)

dynamics = [
    -x + x * y,
    -y
]

# This will work for any arbitrary variable name, but functions will still be printed as f(x0,...,xn)

This system has an equilibrium point at the origin $\mathbf{x} = 0$. We seek to synthesis a Lyapunov function to serve as a certificate of stability of this equilibrium point. This  Lyapunov function will be validated in a region around the equilibrium point. This domain is defined as:

$$ \mathcal{D}  = \{\mathbf{x}: \mathbf{x} < r^2 \} $$

for some user-set parameter r.
<a id='lyap-synth'></a>

In [15]:

r = 10
mode = "l"
numerical_V, symbolic_V = synthesise(dynamics, mode, R=r)

Single Equilibrium point found: 
 [(0, 0)]
0
  learner   0
0 - loss: 13.854019165039062 - acc: 74.3 %
  regulariser   0
  verifier   0
Counterexample Found: [x1 = 4, x0 = 8]
V(ctx) =  527318633535274156421916765176100117624784037618539/7812500000000000000000000000000000000000000000000
Vdot(ctx) =  9504556949213582786472330024138420584498279605629/3906250000000000000000000000000000000000000000000
  trajectoriser   0
  learner   1
0 - loss: 10810.9521484375 - acc: 96.56488549618321 %
100 - loss: 0.9592554569244385 - acc: 96.08778625954199 %
200 - loss: 0.340265691280365 - acc: 96.94656488549619 %
300 - loss: 0.058272816240787506 - acc: 98.37786259541984 %
400 - loss: 0.002147993538528681 - acc: 99.71374045801527 %
500 - loss: -9.580012556398287e-05 - acc: 99.80916030534351 %
  regulariser   1
  verifier   1
Counterexample Found: [x1 = 1, x0 = 9]
V(ctx) =  171998750768996795696943979599695414955364730293428111/100000000000000000000000000000000000000000000000000000
Vdot(ctx) =  12497857754

In [ ]:
print_f(symbolic_V)

### Barrier Cerificates

We now demonstrate synthesising a barrier certificate. Consider the following dynamical system:

$$\begin{cases}
\dot{x} = e^{-x} + y - 1 \\
\dot{y} = - \sin^2 x
\end{cases}$$

We can initialise this dynamical system as follows, with with $x_0 = x $ and $x_1 = y$.

In [11]:
N_Dimensions = 2

x0, x1 = initialise_states(N_Dimensions)

dynamics = [
    exp(-x0) + x1 - 1,
    -sin(x0)**2
]

#### Domain Initialisation
We define the overall domain as 
$$\begin{align*}
& X = \{-2 \leq x, -5 \leq y, \ x \leq 5, \ y  \leq 2 \} \\ 
& X_0 = \{(x+0.5)^2+(y - 0.5)^2 \leq 0.16 \} \\ 
& X_u  = \{ (x - 0.7)^2+(y+0.7)^2 \leq 0.09 \}
\end{align*}$$


This can be initialised as follows:

In [28]:

xd = Rectangle([-2,-5],[5, 2])
xi = Sphere([-0.5, 0.5], 0.4)
xu = Sphere([0.7, -0.7], 0.3)

A barrier function is then synthesised as:
<a id='barrier-synth'></a>

In [ ]:
mode = 'b'
f_n, f_v, = synthesise(dynamics, mode, XD=xd, XI=xi, XU=xu, CEGIS_PARAMETERS={'VERIFIER':VerifierType.DREAL, 
                                                                 })